Utilising a Python script to investigate deprivation locations and their relationships to football violence in England

First, we must import the external packages that we will be using to complete our script. The external packages that we are using are going to allow us to complete 
the desired functions for our analysis. 

In [ ]:
import pandas as pd
import folium
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import rasterio as rio
import rasterio.features
import numpy as np
from matplotlib.patches import Patch